# **Semantic Textual Similarity Model**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

In [3]:
import pickle

# saving
#with open('tokenizer.pickle', 'wb') as handle:
    #pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# loading
with open('/content/drive/MyDrive/Youtube-semantice-search/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [4]:
from keras import backend as K

In [5]:

def exponent_neg_manhattan_distance(left, right):
    ''' Helper function for the similarity estimate of the LSTMs outputs'''
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))


In [6]:
max_seq_length=30

In [7]:
from keras.models import Model
from keras.optimizers import Adadelta

In [8]:
from keras.models import load_model
malstm=load_model('/content/drive/MyDrive/Youtube-semantice-search/quora.h5')

In [9]:
malstm.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 30)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 30)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 30, 200)      17223800    input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 50)           50200       embedding[0][0]              

# **Collecting Timestamps**

In [10]:
!pip install youtube-transcript-api

In [11]:
#drive.mount("/content/drive", force_remount=True)

In [12]:
#!pip install --upgrade pip

In [13]:
from youtube_transcript_api import YouTubeTranscriptApi

#data=YouTubeTranscriptApi.get_transcript('pQJvyMIvW8s')

In [14]:
'''time=[]
transcripts=[]
for values in data:
    for key,value in values.items():
        if(key=="text"):
            transcripts.append(value)
        if(key=="start"):
            time.append(value)'''
        

'time=[]\ntranscripts=[]\nfor values in data:\n    for key,value in values.items():\n        if(key=="text"):\n            transcripts.append(value)\n        if(key=="start"):\n            time.append(value)'

In [15]:
#transcripts

In [16]:
#time

In [17]:
#q1 = tokenizer.texts_to_sequences(np.array(transcripts))
#q1 = pad_sequences(q1, maxlen = 30, padding='post')

In [18]:
#q3 = tokenizer.texts_to_sequences(np.array(["Traffic"]))
#q3 = pad_sequences(q3, maxlen = 30, padding='post')

In [19]:
#p=[]

In [20]:
'''for i in range(30):
    p.append(malstm.predict([q3[0],q1[i]]))'''

'for i in range(30):\n    p.append(malstm.predict([q3[0],q1[i]]))'

In [21]:
'''s=[]
print(q3[0])
for i in range(len(p)):
    if(np.mean(np.array(p[i]))>0.1):
        print(transcripts[i],np.mean(np.array(p[i])))
        s.append(np.mean(np.array(p[i])))'''

's=[]\nprint(q3[0])\nfor i in range(len(p)):\n    if(np.mean(np.array(p[i]))>0.1):\n        print(transcripts[i],np.mean(np.array(p[i])))\n        s.append(np.mean(np.array(p[i])))'

In [22]:
#s

In [23]:
'''sort_index = np.argsort(s) 


sort_index=list(sort_index)
sort_index.reverse()'''

'sort_index = np.argsort(s) \n\n\nsort_index=list(sort_index)\nsort_index.reverse()'

In [24]:
#sort_index

In [25]:
#for i in sort_index:
#    print(transcripts[i],time[i])

In [26]:
#youtubeid='pQJvyMIvW8s'

In [27]:
#link='https://youtu.be/'+youtubeid+'?t='+str(round(time[3]))
#print(link)

In [28]:
#!git clone https://github.com/NagendraH/SKINCANCER6.git

# **Making flask App**

In [29]:
!pip install flask-ngrok

In [30]:
youtubeid='pQJvyMIvW8s'

In [82]:
from flask import Flask
from flask import Flask, redirect, url_for
from flask_ngrok import run_with_ngrok
from flask import render_template
from flask import request
import h5py
from flaskext.mysql import MySQL

app = Flask(__name__)

mysql = MySQL()
app.config['MYSQL_DATABASE_USER'] = "u9fwcv0v3gajj3ky"
app.config['MYSQL_DATABASE_PASSWORD'] = "qzgorFn5oXvosBhJs7t9"
app.config['MYSQL_DATABASE_DB'] = "bff29zhocbsecefuqe3p"
app.config['MYSQL_DATABASE_HOST'] = "bff29zhocbsecefuqe3p-mysql.services.clever-cloud.com"
mysql.init_app(app)
yid1=''
yid=''
temp1=''
run_with_ngrok(app)
'''@app.route('/<string:insert>')
def add(insert):
    sql = "select *from user;"
    conn = mysql.connect()
    cursor =conn.cursor()
    cursor.execute(sql)
    data=cursor.fetchall()
    print(data)
    return render_template('display.html',data=data)'''
    
@app.route("/",methods=['post', 'get'])
def upload_predict():
    l={}
    global yid1
    global yid
    global temp1
    transcripts1=[]
    if request.method == 'POST':
        if request.form.get("submit_b"):
            option = " ".join(map(str,(request.form['options'])))
            print(option)
            print(yid)
            sql = "INSERT INTO user (question1, question2) VALUES (%s, %s);"
            val = (yid,option)
            #print(s1[i])
            conn = mysql.connect()
            cursor =conn.cursor()
            cursor.execute(sql, val)
            conn.commit()
            sql = "select *from user;"
            conn = mysql.connect()
            cursor =conn.cursor()
            cursor.execute(sql)
            temp=cursor.fetchall()
            print(temp)
        

        if request.form.get("submit_a"):
            data=YouTubeTranscriptApi.get_transcript('pQJvyMIvW8s')
            time1=[]
            transcripts1=[]
            for values in data:
                for key,value in values.items():
                    if(key=="text"):
                        transcripts1.append(value)
                    if(key=="start"):
                        time1.append(value)
        if(request.form.get('gsearch1')!=None):
            #print(yid1)
            yid1=str(request.form.get('gsearch1'))
            temp1='https://www.youtube.com/embed/'+yid1[24:]
        if(request.form.get('gsearch2')!=None):
            yid = request.form.get('gsearch2')
            #print(yid)
            data=YouTubeTranscriptApi.get_transcript(str(yid1[24:]))
            time=[]
            transcripts=[]
            for values in data:
                for key,value in values.items():
                    if(key=="text"):
                        transcripts.append(value)
                    if(key=="start"):
                        time.append(value)
            q1 = tokenizer.texts_to_sequences(np.array(transcripts))
            q1 = pad_sequences(q1, maxlen = 30, padding='post')
            q3 = tokenizer.texts_to_sequences(np.array([str(yid)]))
            q3 = pad_sequences(q3, maxlen = 30, padding='post')
            p=[]
            for i in range(30):
                p.append(malstm.predict([q3[0],q1[i]]))
            s=[]
            print(q3[0])
            for i in range(len(p)):
                if(np.mean(np.array(p[i]))>0.1):
                    #print(transcripts[i],np.mean(np.array(p[i])))
                    s.append(np.mean(np.array(p[i])))
            sort_index = np.argsort(s) 
            sort_index=list(sort_index)
            sort_index.reverse()
            l={}
            for i in sort_index[:4]:
                l[transcripts[i]]=(temp1+'?autoplay=1&start='+str(round(time[i])))
                
    print(temp1)
    return render_template('index.html', result = l,id=temp1,transcripts1=transcripts1)
    

if __name__=="__main__":
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://9f5d76da7ad3.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


In [32]:
#from google.colab import drive
#drive.mount('/content/drive')

In [33]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


# **Connecting with the database and training data from database**


In [34]:
!pip install flask-mysql

     |████████████████████████████████| 51kB 6.5MB/s 


In [35]:
from flaskext.mysql import MySQL

app = Flask(__name__)
mysql = MySQL()
app.config['MYSQL_DATABASE_USER'] = "u9fwcv0v3gajj3ky"
app.config['MYSQL_DATABASE_PASSWORD'] = "qzgorFn5oXvosBhJs7t9"
app.config['MYSQL_DATABASE_DB'] = "bff29zhocbsecefuqe3p"
app.config['MYSQL_DATABASE_HOST'] = "bff29zhocbsecefuqe3p-mysql.services.clever-cloud.com"
mysql.init_app(app)

In [36]:
conn = mysql.connect()
cursor =conn.cursor()

In [37]:



query = "Alter TABLE user modify question1  VARCHAR(2000),modify question2 VARCHAR(2000);"
cursor.execute(query)

0

In [38]:
query = "show tables;"
cursor.execute(query)

1

In [39]:
print(cursor)

In [ ]:
for row in cursor:
  print(row)

('user',)


In [ ]:
s1[0]

'How do I structure a big PHP project?'

In [ ]:
s2[0]

'What is the best way to structure a small PHP project and why?'

In [ ]:
for i in range(2,30):
    sql = "INSERT INTO user (question1, question2) VALUES (%s, %s);"
    val = (s1[i],s2[i])
    print(s1[i])
    conn = mysql.connect()
    cursor =conn.cursor()
    cursor.execute(sql, val)
    conn.commit()

What should I say when a girl says I'm "hot"?
Is it safe to give a dog Benadryl to help it sleep?


In [ ]:
conn.commit()

In [ ]:
sql = "select *from user;"
conn = mysql.connect()
cursor =conn.cursor()
cursor.execute(sql)
data=cursor.fetchall()

In [ ]:
print(data)

(('How do I structure a big PHP project?', 'What is the best way to structure a small PHP project and why?'), ('Is it safe to give a dog Benadryl to help it sleep?', 'Is Benadryl safe for consumption by dogs?'), ('Is it safe to give a dog Benadryl to help it sleep?', 'Is Benadryl safe for consumption by dogs?'), ('Organic Chemistry: How can I learn the order of the -I,+I, +m,-m effect easily?', 'I m poor in organic chemistry; how should I study chemistry?'), ('Why do we fix one gear in epicyclic gear train for power input?', 'Does Antarctica have any geopolitical importance?'), ('I have an IQ of 130. Will that increase as I get older?', 'What are the best aspects of living in Townsville, Queensland, Australia as a young adult?'), ('Work culture in sensen networks?', "Rank actor's performances in Suicide Squad."), ('What is the scope of dgital marketing?', 'What is the scope of marketing?'), ('What are the general materials and the requirements for a black hole, is it caused by pressure

In [ ]:
print(len(data))

31


In [ ]:
new_s1=[]
new_s2=[]
for i in range(31):
    new_s1.append(data[i][0])
    new_s2.append(data[i][1])

In [ ]:
new_train_q1 = tokenizer.texts_to_sequences(np.array(new_s1).astype(str))
new_train_q1 = pad_sequences(new_train_q1, maxlen = 30, padding='post')

# X_train_q2 = tokenizer.texts_to_sequences(np.array(listq2))
new_train_q2 = tokenizer.texts_to_sequences(np.array(new_s2).astype(str))
new_train_q2 = pad_sequences(new_train_q2, maxlen = 30, padding='post')

In [ ]:
word_index = tokenizer.word_index

In [ ]:
embedding_index = {}
with open('/content/drive/MyDrive/Youtube-semantice-search/glove.6B.200d.txt','r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:], 'float32')
        embedding_index[word] = vectors
    f.close()

In [ ]:
embedding_matrix = np.random.random((len(word_index)+1, 200))
for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:

def exponent_neg_manhattan_distance(left, right):
    ''' Helper function for the similarity estimate of the LSTMs outputs'''
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

In [ ]:
malstm.compile(loss='mean_squared_error', optimizer=Adadelta(clipnorm=gradient_clipping_norm), metrics=['accuracy'])

In [ ]:
Y_train=np.array([1]*31)

In [ ]:
malstm_trained = malstm.fit([new_train_q1,new_train_q2],Y_train, batch_size=batch_size, epochs=n_epoch)


In [ ]:
p=[]
for i in range(31):
    p.append(np.mean(malstm.predict([new_train_q1[i],new_train_q2[i]])))

In [ ]:
print(p)